In [1]:
import pandas as pd

In [2]:
access = pd.read_csv("/home/jaime/projet_info_3A/private/access.csv",header=0)
id_api = access["Identifiant"][0]
key_api = access["Clé secrète"][0]

In [3]:
import requests 
import os

In [4]:
import emploi_store

In [5]:
client_id = id_api
client_secret = key_api
access_tokens = {}
cached_packages = None
scope = 'api_offresdemploiv2'

In [6]:
# from https://github.com/Cocorico84/employme/blob/855cfcda2c54f42b9c42dcbf4ac87cdf09248b7f/back/manager.py

In [7]:
import requests


url = "https://api.emploi-store.fr/partenaire"

In [8]:
from datetime import datetime, timedelta
import time

#idea from https://stackoverflow.com/questions/42213427/change-variable-after-a-specific-amount-of-time-python
class TimedValue:

    def __init__(self):
        self._started_at = datetime.fromtimestamp(0) # to do False if not started

        pass
    
    def start(self):
        self._started_at = datetime.utcnow()
        

    def __call__(self):
        time_passed = datetime.utcnow() - self._started_at
        if time_passed.total_seconds() < 4.5:
            return True
        return False

    
# value = False


In [9]:
value = TimedValue()

def set_global_TOKEN():
    global TOKEN
    if value()==False :
            url_token = 'https://entreprise.pole-emploi.fr/connexion/oauth2/access_token'
            r = requests.post(url_token, headers={'Content-Type': 'application/x-www-form-urlencoded'},
                              params={'realm': '/partenaire'},
                              data={'grant_type': 'client_credentials', 'client_id': id_api,
                                    'client_secret': key_api,
                                    'scope': f'application_{id_api} api_offresdemploiv2 o2dsoffre'})
            data = r.json()
            TOKEN = data['access_token']
            value.start()

In [10]:
for i in range(10):
    time.sleep(0.75)
    set_global_TOKEN()
    print(TOKEN)

5e8b4b61-4ff9-4389-a4a8-47cf96b4054c
5e8b4b61-4ff9-4389-a4a8-47cf96b4054c
5e8b4b61-4ff9-4389-a4a8-47cf96b4054c
5e8b4b61-4ff9-4389-a4a8-47cf96b4054c
5e8b4b61-4ff9-4389-a4a8-47cf96b4054c
5e8b4b61-4ff9-4389-a4a8-47cf96b4054c
e69427eb-ac4a-4e54-8933-386bde4191d0
e69427eb-ac4a-4e54-8933-386bde4191d0
e69427eb-ac4a-4e54-8933-386bde4191d0
e69427eb-ac4a-4e54-8933-386bde4191d0


In [11]:
def connection(range_str='0-149'):
    set_global_TOKEN()
    url_offers = f'{url}/offresdemploi/v2/offres/search'
    r = requests.get(url=url_offers,params={'range':range_str},   #{'motsCles': 'informatique'} ,# params={'rantge=':range_str}, #,  params={'motsCles': 'informatique'}
        headers={'Authorization': f'Bearer {TOKEN}'})
    data = r.json()

    return data

In [12]:
def get_results_as_df(range_str='0-149'):
    
    request_as_dict=connection(range_str)

    df = pd.DataFrame.from_dict(request_as_dict['resultats'])
    
#     for col_as_list in ['lieuTravail','origineOffre','partenaires']: #, 'competences', 'contact','entreprise','qualitesProfessionnelles','salaire']:
#         if col_as_list in df.columns:
#             df = pd.concat([df.drop([col_as_list], axis=1), df[col_as_list].apply(pd.Series)], axis=1)
    
    return df #request_as_dict['filtres possibles']


In [13]:
data = connection()

In [14]:
iteration_index = list(range(0,1000,150))

In [16]:
# list_of_results = [connection(
#     str(str(min(i,1000))+'-'+str(min(i+149,1149)))) for i in list(range(0,1150,150))]

# list_dfs = [pd.DataFrame.from_dict(request_as_dict['resultats']) for request_as_dict in list_of_results]

# all_dfs = pd.concat(list_dfs)

# for col_as_list in ['lieuTravail','origineOffre','partenaires', 'competences']: #, 'contact','entreprise','qualitesProfessionnelles','salaire']:
#     if col_as_list in all_dfs.columns:
#         all_dfs = pd.concat([all_dfs.drop([col_as_list], axis=1), all_dfs[col_as_list].apply(pd.Series)], axis=1)

In [17]:
# all_dfs = all_dfs.astype(str).drop_duplicates()

# all_dfs.to_csv("df_max_pages.csv", sep=",")

# loop through creation date

In [30]:
def connection_with_dates(range_str, min_date, max_date):
#     time.sleep(1) # to avoid <Response [429]>, too many requests (if JSONDecodeError)
    str_min_date = min_date.strftime("%Y-%m-%dT%H:%M:%SZ") 
    str_max_date = max_date.strftime("%Y-%m-%dT%H:%M:%SZ")
    
    set_global_TOKEN()
    url_offers = f'{url}/offresdemploi/v2/offres/search'
    r = requests.get(url=url_offers,params={'range':range_str,'minCreationDate':str_min_date,'maxCreationDate':str_max_date},   #{'motsCles': 'informatique'} ,# params={'rantge=':range_str}, #,  params={'motsCles': 'informatique'}
        headers={'Authorization': f'Bearer {TOKEN}'})
#     print(r)
    try:
        data = r.json()
    except json.decoder.JSONDecodeError:
        
        try:
            time.sleep(float(r.headers['Retry-After'])+0.1) # to avoid <Response [429]>, too many requests (if JSONDecodeError).
        except :
            print("###")
            print(str_min_date,str_max_date)
            print(r)
            print(r.headers)
#             data = connection_with_dates(range_str,min_date,max_date) # to comment ?

        data = connection_with_dates(range_str,min_date,max_date)
        
    return data    

In [31]:
all_dfs=[]

In [1]:
path="/home/jaime/projet_info_3A/private/day_by_day/"

In [64]:
num_days_lookback = 365*2
dates_range = [(datetime.utcnow()-timedelta(days=i)-timedelta(days=423)) for i in np.linspace(0,num_days_lookback,num_days_lookback)]

In [65]:
datetime.utcnow()-timedelta(days=423)

datetime.datetime(2019, 2, 9, 5, 50, 7, 644747)

In [66]:
# dates_range = [(datetime.utcnow()-timedelta(days=i)).strftime("%Y-%m-%dT%H:%M:%SZ") for i in range(30)]

In [67]:
def create_date_range_str(i):
    return str(str(min(i,1000))+'-'+str(min(i+149,1149)))

In [68]:
def cut_datime_range_into_frecs(min_day,max_day, num_items):
    # mettre num items au lieu de 10 dans la ligne au dessous
    timestamp_range = np.linspace(datetime.timestamp(max_day), datetime.timestamp(min_day), 10, dtype=np.int64)
    timestamp_range = [datetime.fromtimestamp(i) for i in timestamp_range]
    return timestamp_range
    

In [69]:
def query_results(min_day,max_day):
    list_of_results= []
    
    for i in list(range(0,1150,150)):
        date_range_str = create_date_range_str(i)
        res = connection_with_dates(date_range_str,min_day,max_day)
        list_of_results.append(res)
        
        try :
            if len(res['resultats'])<150:
                break
        except:
            pass

    return list_of_results

In [70]:
def get_results_in_dates_list(dates_range):
    
    for day in range(1,len(dates_range)-1):
        # IMPORTANT: check if range(1,len(dates_range)-1) or range(1,len(dates_range))

        min_day = dates_range[day]
        max_day = dates_range[day-1]

        # do it more efficient (function and if condition on results)
#         list_of_results = [connection_with_dates(create_date_range_str(i),min_day,max_day)
#                            for i in list(range(0,1150,150))]

        list_of_results = query_results(min_day,max_day)
    
        try: 
            list_dfs = [pd.DataFrame.from_dict(request_as_dict['resultats']) for request_as_dict in list_of_results]
        except KeyError:
            print("AAAAAAA")
            print("KEY ERROR, CHECK GLOBAL VAR saving_error")
            global saving_error
            saving_error=list_of_results
            
        dfs_date = pd.concat(list_dfs,sort=True)

        if len(dfs_date.index) == 0:
            pass

        elif len(dfs_date.index) < 1200:
            csv_name=os.path.join(path, str(dates_range[day+1])+".csv")
            dfs_date.to_csv(csv_name, sep=",")
            # return ?

        if len(dfs_date.index) == 1200:
            print("start missing part of the day")
            # does not save csv, cuts time range and reruns code
            new_range = cut_datime_range_into_frecs(min_day,max_day,5)
            get_results_in_dates_list(new_range)
            # do something to avoid infinite loop (i.e. if more than 1200 at the same day, hour,second ?)
            print("end missing part of the day: SOLVED")


In [71]:
get_results_in_dates_list(dates_range)

###
2018-09-28T01:25:25Z 2018-09-29T01:27:23Z
<Response [502]>
{'Access-Control-Expose-Headers': 'Content-Range', 'Date': 'Tue, 07 Apr 2020 05:52:01 GMT', 'Strict-Transport-Security': 'max-age=64281600; includeSubDomains', 'X-Frame-Options': 'ALLOW-FROM https://*.pole-emploi.fr', 'Set-Cookie': 'TS01600790=01b3abf0a2f6a3f6b5559653c856fa4dd6a5f8feb3f068a34f6d85386c8a318a3cee27916ac2e2483cdb40e53639db24bfaf156a5d; Path=/; Secure; HTTPOnly', 'Vary': 'Accept-Encoding', 'Content-Encoding': 'deflate', 'Transfer-Encoding': 'chunked'}
###
2018-05-12T20:52:49Z 2018-05-13T20:54:48Z
<Response [502]>
{'Access-Control-Expose-Headers': 'Content-Range', 'Date': 'Tue, 07 Apr 2020 05:54:15 GMT', 'Strict-Transport-Security': 'max-age=64281600; includeSubDomains', 'X-Frame-Options': 'ALLOW-FROM https://*.pole-emploi.fr', 'Set-Cookie': 'TS01600790=01b3abf0a228aa73fcecfa9edd5a9d83a01852e447d5d7f995a9fd756df703429ae2ccba61f2d3170c3485e311cbad323df90381eb; Path=/; Secure; HTTPOnly', 'Vary': 'Accept-Encoding',

KeyboardInterrupt: 

In [ ]:
#saving_error

# concatenate all dfs

In [72]:
import pandas as pd
import glob

# from https://stackoverflow.com/questions/20906474/import-multiple-csv-files-into-pandas-and-concatenate-into-one-dataframe

path = r'/home/jaime/projet_info_3A/private/day_by_day/' # use your path
all_files = glob.glob(path + "/*.csv")

li = []

for filename in all_files:
    df = pd.read_csv(filename, index_col=None, header=0)
    li.append(df)

# TODO: convert to str and remove duplicates
frame = pd.concat(li, axis=0, ignore_index=True)

/home/jaime/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:16: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  app.launch_new_instance()


In [73]:
frame = frame.astype(str).drop_duplicates()

In [74]:
frame.to_csv("all_offers.csv", sep=",")

In [75]:
len(frame)

275897

In [76]:
subset = frame[:10]

In [77]:
subset

,Unnamed: 0,accessibleTH,agence,alternance,appellationlibelle,competences,complementExercice,conditionExercice,contact,dateActualisation,...,qualificationLibelle,qualitesProfessionnelles,romeCode,romeLibelle,salaire,secteurActivite,secteurActiviteLibelle,trancheEffectifEtab,typeContrat,typeContratLibelle
0,0,False,nan,False,Conducteur / Conductrice de véhicules Super Lo...,"[{'exigence': 'E', 'libelle': 'carte conducteu...",nan,nan,{'coordonnees1': 'Courriel : cv-fpotin@sbc-int...,nan,...,Employé qualifié,nan,N4101,Conduite de transport de marchandises sur long...,"{'libelle': 'Horaire de 10,15 Euros sur 12 mois'}",78.0,Activités des agences de travail temporaire,3 à 5 salariés,MIS,Mission intérimaire - 6 Mois
1,1,False,{'courriel': 'entreprise.pic0010@pole-emploi.n...,False,Chef d'équipe gros oeuvre,"[{'code': '118365', 'exigence': 'S', 'libelle'...",nan,nan,{'coordonnees1': '11 RUE Paul Vimereu\nCS80704...,nan,...,"Ouvrier qualifié (P3,P4,OHQ)",[{'description': 'Capacité à faire des choix p...,F1202,Direction de chantier du BTP,{'libelle': 'Annuel de 23000 Euros à 40000 Eur...,78.0,Activités des agences de travail temporaire,nan,CDI,Contrat à durée indéterminée
2,2,False,nan,False,Ingénieur / Ingénieure génie civil,"[{'code': '118773', 'exigence': 'S', 'libelle'...",nan,nan,"{'coordonnees1': 'Courriel : ote.rh@ote.fr', '...",nan,...,Cadre,[{'description': 'Capacité à travailler et à s...,F1106,Ingénierie et études du BTP,"{'complement1': 'Chèque repas', 'complement2':...",71.0,"Ingénierie, études techniques",100 à 199 salariés,CDI,Contrat à durée indéterminée
3,3,False,{'courriel': 'ale.le-puy@pole-emploi.fr'},True,Conseiller commercial professionnel / Conseill...,"[{'code': '101563', 'exigence': 'S', 'libelle'...",nan,nan,{'coordonnees1': '5 RUE DES CHEVALIERS ST JEAN...,nan,...,Ouvrier spécialisé,nan,C1206,Gestion de clientèle bancaire,"{'commentaire': ""contrat d'apprentissage"", 'co...",71.0,"Ingénierie, études techniques",0 salarié,CDD,Contrat à durée déterminée - 24 Mois
4,4,False,nan,False,Menuisier / Menuisière,"[{'code': '124985', 'exigence': 'S', 'libelle'...",nan,nan,{'coordonnees1': 'Courriel : immigrationaquebe...,nan,...,Employé qualifié,nan,H2206,Réalisation de menuiserie bois et tonnellerie,"{'commentaire': '16$CAD / heure', 'complement1...",78.0,Activités des agences de placement de main-d'o...,0 salarié,CDD,Contrat à durée déterminée - 24 Mois
5,5,False,nan,False,Conducteur / Conductrice de véhicules Super Lo...,"[{'exigence': 'E', 'libelle': 'carte conducteu...",nan,nan,{'coordonnees1': 'Courriel : cv-fpotin@sbc-int...,nan,...,"Ouvrier qualifié (P3,P4,OHQ)",nan,N4101,Conduite de transport de marchandises sur long...,"{'libelle': 'Horaire de 10,21 Euros sur 12 mois'}",78.0,Activités des agences de travail temporaire,3 à 5 salariés,MIS,Mission intérimaire - 6 Mois
6,6,False,{'courriel': 'recrutementgolfesttropez@pole-em...,False,Serveur / Serveuse en restauration,"[{'code': '104323', 'exigence': 'S', 'libelle'...",nan,nan,"{'coordonnees1': 'CS 80083 98 RUE CARNOT', 'co...",nan,...,Employé qualifié,nan,G1803,Service en restauration,"{'commentaire': '1900€ nets / à négocier', 'co...",56.0,Restauration traditionnelle,10 à 19 salariés,CDD,Contrat à durée déterminée - 9 Mois
7,7,False,nan,False,Coiffeur / Coiffeuse,"[{'code': '101901', 'exigence': 'S', 'libelle'...",nan,nan,{'coordonnees1': 'Courriel : immigrationaquebe...,nan,...,Employé qualifié,"[{'description': 'Capacité à planifier, à prio...",D1202,Coiffure,"{'commentaire': '12 $CAD / h +pourboires', 'co...",78.0,Activités des agences de placement de main-d'o...,0 salarié,CDI,Contrat à durée indéterminée
8,8,False,nan,False,Mécanicien / Mécanicienne poids lourds,"[{'code': '107046', 'exigence': 'S', 'libelle'...",nan,nan,{'coordonnees1': 'Courriel : immigrationaquebe...,nan,...,Employé qualifié,nan,I1604,Mécanique automobile et entretien de véhicules,"{'commentaire': 'à partir de 25$CAD/hr', 'comp...",78.0,Activités des agences de placement de main-d'o...,0